In [ ]:
import keras
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Dense, Flatten, LeakyReLU, Input, BatchNormalization, Reshape, Conv2D, Conv2DTranspose
from keras.optimizers import Adam, SGD
from keras import Model, Sequential
import os
import cv2

In [ ]:
class GANS():
    def __init__(self):
        keras.backend.clear_session()
        self.img_rows = 100
        self.img_cols = 100
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        
        optimizer = Adam(0.0002, 0.8)
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss = 'binary_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
        
        self.generator = self.build_generator()
        self.generator.compile(loss = 'binary_crossentropy', optimizer = optimizer)
        
        z = Input(shape = (300, ))
        img = self.generator(z)
        
        
        self.discriminator.trainable = False
        
        #combined model will disallow trained values from the discriminator
        valid = self.discriminator(img)
        
        
        self.combined = Model(z, valid)
        self.combined.compile(loss= 'binary_crossentropy', optimizer = optimizer)
        
    def build_discriminator(self):
        img_shape = (self.img_rows, self.img_cols, self.channels)
        
        model = Sequential()
        
        model.add(Flatten(input_shape = img_shape))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(Dense(1, activation = 'sigmoid'))
        model.summary()
        
        img = Input(shape = img_shape)
        validity = model(img)
        return Model(img, validity)
    
    def build_generator(self):
        
        noise_shape = (4, )
        model = Sequential()
        
        model.add(Dense(128, input_shape = noise_shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum = 0.8))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(BatchNormalization(momentum = 0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(BatchNormalization(momentum = 0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(BatchNormalization(momentum = 0.8))
        model.add(Dense(np.prod(self.img_shape), activation = 'tanh'))
        model.add(Reshape(self.img_shape))
        model.summary()

        noise = Input(shape = noise_shape)
        img = model(noise)
        return Model(noise, img)
    
    def train(self, epochs, batch_size = 16, save_interval=50):
        #load the data
        X_train = self.process_images()
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)

        #rescale img data to -1 to 1
        half_batch = int(batch_size/2)
        
        for epoch in range(epochs):
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            imgs = X_train[idx]
            imgs = imgs.reshape(8, 100,100, 3)
            noise = np.random.normal(0, 1, (half_batch, 300))
            
            gen_imgs = self.generator.predict(noise)
            gen_imgs = gen_imgs.reshape(8, 100, 100, 3)
            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            
            #train generator
            
            noise = np.random.normal(0, 1 , (batch_size, 300))
            
            valid_y = np.array([1] * batch_size)
            
            g_loss = self.combined.train_on_batch(noise, valid_y)
            
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))            
            
            if epoch % save_interval == 0:
                self.save_imgs(epoch)
    def save_imgs(self, epoch):
        noise = np.random.normal(0, 1, (1, 300, ))
        gen_imgs = self.generator.predict(noise)
        #print(gen_imgs)
        gen_imgs = (gen_imgs + 0.5) * 255
        
        cv2.imwrite("img\\img_" + str(epoch) + ".jpg", gen_imgs[0])
        
        plt.close()
    
    def process_images(self):
        imgs = []
        path = "flower_images\\"
        for i in os.listdir(path):
            im = cv2.imread(path + i)
            im = cv2.resize(im, (100, 100))
            imgs.append(im)
        imgs = np.asarray(imgs) 
        return imgs
    
if __name__ == '__main__':
    gan = GANS()
    gan.train(epochs = 1000000, batch_size = 16, save_interval =100)
